[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ryansteed/image-gpt-bias/blob/master/notebooks/experiments.ipynb)

# Image GPT Bias
**Image Embedding Association Test**

Ryan Steed

This script adapted from https://colab.research.google.com/github/apeguero1/image-gpt/blob/master/Transformers_Image_GPT.ipynb.


In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath('..'))

%load_ext autoreload
%autoreload 2

%cd /Users/steed/caliskan/git/image-gpt-bias

[Errno 2] No such file or directory: '/Users/steed/caliskan/git/image-gpt-bias'
/content


## Download Model

In [2]:
!nvidia-smi #OpenAI says you need 16GB GPU for the large model, but it may work if you lower n_sub_batch on the others.

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [3]:
import urllib

# don't share these to Github or elsewhere!
u = "your_username"
p = urllib.parse.quote("your_password")
!git clone --recurse-submodules -j8 https://$u:$p@github.com/$u/image-gpt-bias.git
%cd image-gpt-bias

fatal: destination path 'image-gpt-bias' already exists and is not an empty directory.
/content/image-gpt-bias


In [4]:
# note - you may need to restart the kernel for these installations to take effect
!pip install -e .
!pip install -e weat

Obtaining file:///content/image-gpt-bias
     |████████████████████████████████| 1.0MB 8.1MB/s 
     |████████████████████████████████| 1.1MB 31.8MB/s 
     |████████████████████████████████| 890kB 51.1MB/s 
     |████████████████████████████████| 3.0MB 42.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=39f329199919716d85b4b368f2f367d7302eac9cc8fbdabedb3fbb772ed215de
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
  Running setup.py develop for ieat
Obtaining file:///content/image-gpt-bias/weat
  Running setup.py develop for weat


In [ ]:
model_sizes = ["s", "m", "l"] #small medium large, xl not available
model_size = "l"
models_dir = "models"
color_clusters_dir = "clusters"
n_px = 32

In [ ]:
# download the model - skip if already downloaded
!python image-gpt/download.py --model {model_size} --ckpt 1000000 --clusters --download_dir {models_dir}/{model_size}
!python image-gpt/download.py --clusters --download_dir {color_clusters_dir}

## IATs

### Quick Summary

In [4]:
from ieat.api import test_all

import pandas as pd
import numpy as np

results = test_all(
    {
        'simclr': (50,1,0)
    },
    tests=("Insect-Flower")
)

09/22 07:31:44 PM: Extracting images from data/experiments/insect-flower/flower
09/22 07:31:44 PM: resolver HttpCompressedFileResolver does not support the provided handle.
09/22 07:31:44 PM: resolver GcsCompressedFileResolver does not support the provided handle.


## Insect-Flower ##
# simclr #
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


09/22 07:31:49 PM: Saver not created because there are no variables in the graph to restore
100%|██████████| 15/15 [00:00<00:00, 26.32it/s]
09/22 07:32:31 PM: Extracting images from data/experiments/insect-flower/insect
100%|██████████| 20/20 [00:00<00:00, 41.32it/s]


ValueError: ignored

In [ ]:
  import pandas as pd

results_df = pd.DataFrame(results).transpose()
results_df.columns = ["X", "Y", "A", "B", "effect", "p", "n_t", "n_a"]
for c in results_df.columns[:4]:
    results_df[c] = results_df[c].str.split("/").str[-1]
results_df["sig"] = ""
for l in [0.10, 0.05, 0.01]:
    results_df.sig[results_df.p < l] += "*"
results_df[results_df.index.get_level_values(1) == "openai"]
# results_df

,,X,Y,A,B,effect,p,n_t,n_a,sig
Insect-Flower,openai,flower,insect,pleasant,unpleasant,0.344033,0.0733,35,55,*
Weapon,openai,white,black,tool,weapon,0.861729,0.0681818,6,7,*
Weapon (Modern),openai,white,black,tool-modern,weapon-modern,0.876497,0.0606061,6,9,*
Native,openai,euro,native,us,world,-0.334465,0.7315,8,5,
Asian,openai,european-american,asian-american,american,foreign,0.254383,0.33658,6,6,
Weight,openai,thin,fat,pleasant,unpleasant,1.66548,0.0001,10,55,***
Skin-Tone,openai,light,dark,pleasant,unpleasant,1.25532,0.00699301,7,55,***
Disability,openai,disabled,abled,pleasant,unpleasant,-0.0185307,0.528571,4,55,
President - Kennedy vs. Trump,openai,kennedy,trump,pleasant,unpleasant,0.0506318,0.469697,6,55,
President - B. Clinton vs. Trump,openai,clinton,trump,pleasant,unpleasant,-0.868112,0.937229,6,55,


In [ ]:
# Generate latex table output
experiments = (
    "Insect-Flower",
    "Gender-Science",
    "Gender_Career",
    "Skin-Tone",
    "Race",
    "Weapon",
    "Weapon (Modern)",
    "Native",
    "Asian",
    "Weight",
    "Religion",
    "Sexuality",
    "Disability",
    "Arab-Muslim",
    "Age"
)
results_df.index = results_df.index.set_names(["Test", "Model"])
results_df[results_df.index.get_level_values("Test").isin(experiments)]\
    .drop(columns=["X", "Y", "A", "B"]).to_latex('output/results.tex')
results_df.droplevel("Model").groupby("Test").head(1)\
    .drop(columns=["effect", "p", "n_t", "n_a", "sig"]).to_latex('output/iats.tex')
stimuli = pd.read_csv("data/stimuli.csv")
stimuli.dropna(axis=0, subset=["Word", "Source"])\
    .drop(columns="Collected (N=n)").set_index(["IAT", "Category"]).to_latex("output/stimuli.tex")

### Detail View

#### Weapons

In [ ]:
from ieat.api import test

test(
    "data/experiments/weapon/black",
    "data/experiments/weapon/white",
    "data/experiments/weapon/tool",
    "data/experiments/weapon/weapon",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px
#     from_cache=False
)

#### Native

In [ ]:
test(
    "data/experiments/native/euro",
    "data/experiments/native/native",
    "data/experiments/native/us",
    "data/experiments/native/world",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px
#     from_cache=False
)

#### Asian

In [ ]:
test(
    "data/experiments/asian/european-american",
    "data/experiments/asian/asian-american",
    "data/experiments/asian/american",
    "data/experiments/asian/foreign",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px
#     from_cache=False
)

#### Insect-Flower

In [ ]:
from ieat.api import test

test(
    "data/experiments/insect-flower/flower",
    "data/experiments/insect-flower/insect",
    "data/experiments/valence/pleasant",
    "data/experiments/valence/unpleasant",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px,
#     from_cache=False,
    verbose=True
)

#### Weight

In [ ]:
from ieat.api import test

test(
    "data/experiments/weight/thin",
    "data/experiments/weight/fat",
    "data/experiments/valence/pleasant",
    "data/experiments/valence/unpleasant",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px,
#     from_cache=False,
    verbose=True
)

#### Skin-Tone

In [ ]:
from ieat.api import test

test(
    "data/experiments/skin-tone/light",
    "data/experiments/skin-tone/dark",
    "data/experiments/valence/pleasant",
    "data/experiments/valence/unpleasant",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px,
#     from_cache=False,
    verbose=True
)

#### Disability

In [ ]:
from ieat.api import test

test(
    "data/experiments/disabled/disabled",
    "data/experiments/disabled/abled",
    "data/experiments/valence/pleasant",
    "data/experiments/valence/unpleasant",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px,
#     from_cache=False,
    verbose=True
)

#### Presidents

In [ ]:
from ieat.api import test

test(
    "data/experiments/presidents/trump",
    "data/experiments/presidents/kennedy",
    "data/experiments/valence/pleasant",
    "data/experiments/valence/unpleasant",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px,
#     from_cache=False,
    verbose=True
)

In [ ]:
from ieat.api import test

test(
    "data/experiments/presidents/trump",
    "data/experiments/presidents/clinton",
    "data/experiments/valence/pleasant",
    "data/experiments/valence/unpleasant",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px,
#     from_cache=False,
    verbose=True
)

In [ ]:
from ieat.api import test

test(
    "data/experiments/presidents/trump",
    "data/experiments/presidents/bush",
    "data/experiments/valence/pleasant",
    "data/experiments/valence/unpleasant",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px,
#     from_cache=False,
    verbose=True
)

In [ ]:
from ieat.api import test

test(
    "data/experiments/presidents/trump",
    "data/experiments/presidents/lincoln",
    "data/experiments/valence/pleasant",
    "data/experiments/valence/unpleasant",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px,
#     from_cache=False,
    verbose=True
)

#### Religion

In [ ]:
from ieat.api import test

test(
    "data/experiments/religion/christianity",
    "data/experiments/religion/judaism",
    "data/experiments/valence/pleasant",
    "data/experiments/valence/unpleasant",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px,
#     from_cache=False,
    verbose=True
)

#### Gender-Science

In [ ]:
from ieat.api import test

test(
    "data/experiments/gender/science",
    "data/experiments/gender/liberal-arts",
    "data/experiments/gender/male",
    "data/experiments/gender/female",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px,
#     from_cache=False,
    verbose=True
)

#### Gender-Career

In [ ]:
from ieat.api import test

test(
    "data/experiments/gender/career",
    "data/experiments/gender/family",
    "data/experiments/gender/male",
    "data/experiments/gender/female",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px,
#     from_cache=False,
    verbose=True
)

#### Sexuality

In [ ]:
from ieat.api import test

test(
    "data/experiments/sexuality/gay",
    "data/experiments/sexuality/straight",
    "data/experiments/valence/pleasant",
    "data/experiments/valence/unpleasant",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px,
#     from_cache=False,
    verbose=True
)

#### Race

In [ ]:
from ieat.api import test

test(
    "data/experiments/race/african-american",
    "data/experiments/race/european-american",
    "data/experiments/valence/pleasant",
    "data/experiments/valence/unpleasant",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px,
#     from_cache=False,
    verbose=True
)

#### Arab-Muslim

In [ ]:
from ieat.api import test

test(
    "data/experiments/arab-muslim/other-people",
    "data/experiments/arab-muslim/arab-muslim",
    "data/experiments/valence/pleasant",
    "data/experiments/valence/unpleasant",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px,
#     from_cache=False,
    verbose=True
)

#### Age

In [ ]:
from ieat.api import test

test(
    "data/experiments/age/young",
    "data/experiments/age/old",
    "data/experiments/valence/pleasant",
    "data/experiments/valence/unpleasant",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px,
#     from_cache=False,
    verbose=True
)

### Download Cached Embeddings

In [ ]:
# to download from colab
from google.colab import files

!zip -r embeddings_colab.zip embeddings
files.download("embeddings_colab.zip")